# nlp-project-vqa


In [1]:
obs_path = "s3://nlp-haofeng/nlp_project_vqa/" # hf's path
# obs_path = "s3://nlp-haofeng/nlp_project_vqa/" # hy's path
# obs_path = "s3://nlp-haofeng/nlp_project_vqa/" # zc's path
# obs_path = "s3://nlp-haofeng/nlp_project_vqa/" # ct's path

In [2]:
import moxing as mox
mox.file.copy_parallel(src_url=obs_path+"mindrecord", dst_url='./mindrecord') 
mox.file.copy_parallel(src_url=obs_path+"preprocess", dst_url='./preprocess')
mox.file.copy_parallel(src_url=obs_path+"utils",      dst_url='./utils')
mox.file.copy_parallel(src_url=obs_path+"model",      dst_url='./model')
mox.file.copy_parallel(src_url=obs_path+"ckpt",       dst_url='./ckpt')
#mox.file.copy_parallel(src_url=obs_path+"data",       dst_url='./data')

INFO:root:Using MoXing-v2.0.0.rc2.4b57a67b-4b57a67b
INFO:root:Using OBS-Python-SDK-3.20.9.1


In [3]:
import mindspore
import numpy as np
import os
from easydict import EasyDict
from preprocess.preprocess import *
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"  # 允许重复载入lib文件

INFO:matplotlib.font_manager:generated new fontManager


图模式

In [4]:
from mindspore import context
context.set_context(mode=context.GRAPH_MODE)

PyNative模式

In [2]:
from mindspore import context
context.set_context(mode=context.PYNATIVE_MODE)

Ascend 环境安装 MindSpore Hub

In [ ]:
! pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/1.3.0/Hub/any/mindspore_hub-1.3.0-py3-none-any.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple

## 1 预处理

### 1.1 预处理配置

In [5]:
padding = '<pad>'
config = EasyDict({
	'train_img_path': './data/images/train/COCO_train2014_',
	'train_ans_path': './data/annotations/train.json',
	'train_que_path': './data/questions/train.json',
	'valid_img_path': './data/images/val/COCO_val2014_',
	'valid_ans_path': './data/annotations/val.json',
	'valid_que_path': './data/questions/val.json',
	'test_img_path': './data/images/test/COCO_val2014_',
	'test_ans_path':  './data/annotations/test.json',
	'test_que_path':  './data/questions/test.json',
	'max_length': 25,
	'dict_path': './mindrecord/dict.npy',
	'idx_word_dict_path': './mindrecord/idx_word_dict.npy',
	'num_splits': 1,
	'train_mindrecord_path': './mindrecord/train.mindrecord',
	'valid_mindrecord_path': './mindrecord/valid.mindrecord',
	'test_mindrecord_path':  './mindrecord/test.mindrecord',
})

### 1.2 读取数据

注: 只取那些答案长度为1的vqa组合

In [4]:
# get 3 types of input data
train_images, train_questions, train_answers = get_list(config.train_que_path, config.train_ans_path)
valid_images, valid_questions, valid_answers = get_list(config.valid_que_path, config.valid_ans_path)
test_images,  test_questions,  test_answers  = get_list(config.test_que_path,  config.test_ans_path)

In [5]:
total_questions = train_questions + valid_questions + test_questions
total_answers   = train_answers + valid_answers + test_answers

### 1.3 构建词典

In [6]:
# build word vocab
word_dict = dict({'<pad>': 0})
word_dict = add_word_into_dict(total_questions, word_dict)
word_dict = add_word_into_dict(total_answers, word_dict)

In [7]:
# build revert dict
idx_word_dict = dict()
for item in word_dict.items():
	idx_word_dict[item[1]] = item[0]

In [8]:
# save dict
np.save(config.dict_path, word_dict)
np.save(config.idx_word_dict_path, idx_word_dict)

In [ ]:
mox.file.copy_parallel(src_url="./mindrecord/dict.npy", dst_url=obs_path+"mindrecord/dict.npy") 
mox.file.copy_parallel(src_url="./mindrecord/idx_word_dict.npy", dst_url=obs_path+"mindrecord/idx_word_dict.npy") 

### 1.4 向量化 & 补齐长度

In [9]:
# word -> vector & padding
train_questions_vec = get_vec_and_pad(train_questions, word_dict, config.max_length)
valid_questions_vec = get_vec_and_pad(valid_questions, word_dict, config.max_length)
test_questions_vec = get_vec_and_pad(test_questions, word_dict, config.max_length)

train_answers_vec = get_vec_and_pad(train_answers, word_dict, 1)
valid_answers_vec = get_vec_and_pad(valid_answers, word_dict, 1)
test_answers_vec = get_vec_and_pad(test_answers, word_dict, 1)

### 1.5 生成MindRecord

train

In [ ]:
generate_mindrecord(config.train_mindrecord_path, config.train_img_path, config.num_splits, train_images, train_questions_vec, train_answers_vec)

In [ ]:
mox.file.copy_parallel(src_url="./mindrecord/train.mindrecord",    dst_url=obs_path+"mindrecord/train.mindrecord") 
mox.file.copy_parallel(src_url="./mindrecord/train.mindrecord.db", dst_url=obs_path+"mindrecord/train.mindrecord.db") 

valid

In [ ]:
generate_mindrecord(config.valid_mindrecord_path, config.valid_img_path, config.num_splits, valid_images, valid_questions_vec, valid_answers_vec)

In [ ]:
mox.file.copy_parallel(src_url="./mindrecord/valid.mindrecord",    dst_url=obs_path+"mindrecord/valid.mindrecord") 
mox.file.copy_parallel(src_url="./mindrecord/valid.mindrecord.db", dst_url=obs_path+"mindrecord/valid.mindrecord.db") 

test

In [ ]:
generate_mindrecord(config.test_mindrecord_path, config.test_img_path, config.num_splits, test_images, test_questions_vec, test_answers_vec)

In [ ]:
mox.file.copy_parallel(src_url="./mindrecord/test.mindrecord",    dst_url=obs_path+"mindrecord/test.mindrecord") 
mox.file.copy_parallel(src_url="./mindrecord/test.mindrecord.db", dst_url=obs_path+"mindrecord/test.mindrecord.db") 

## 2 加载数据

### 2.1 加载词典

In [7]:
# load dict
word_dict = np.load(config.dict_path, allow_pickle=True).item()
idx_word_dict = np.load(config.idx_word_dict_path, allow_pickle=True).item()

### 2.2 训练配置

In [6]:
model_name = 'baseline'

In [7]:
batch_scale = 'normal'
# batch_scale = 'large'

In [8]:
train_config = EasyDict({
	'model': model_name,
	'vocab_size': 10233,
	'batch_size': 128 if batch_scale == 'large' else 32,
	'epoch_size': 3,
	'max_length': 25,
	'hidden_size': 1024,
	'lr': 2e-5,
	'momentum': 0.8,
	'early_stop': 100,
	# 'save_checkpoint_steps': 1279,
	'ckpt_save_path': './ckpt',
	'checkpoint_path': './ckpt/'+model_name+'.ckpt',
	# 'keep_checkpoint_max': 2,
})

### 2.3 生成数据集

In [9]:
# platform = 'Local'
# platform = 'Colob'
platform = 'Ascend'

In [10]:
num_parallel_workers = 4 if platform == 'Ascend' or platform == 'Local' else 2
print(num_parallel_workers)
# create dataset
train_dataset = generate_dataset(config.train_mindrecord_path, train_config.batch_size, 1, num_parallel_workers)
valid_dataset = generate_dataset(config.valid_mindrecord_path, train_config.batch_size, 1, num_parallel_workers)
test_dataset  = generate_dataset(config.test_mindrecord_path , train_config.batch_size, 1, num_parallel_workers)

4


## 3 训练模型

### 3.1 创建模型

In [11]:
import mindspore.nn as nn
import mindspore.ops.operations as P
from utils.metric_utils import *
from utils.wrapper_utils import *
from utils.callback_utils import *

In [14]:
class Network(nn.Cell):
	def __init__(self, train_config):
		super(Network, self).__init__()
		self.reshape = P.Reshape()
		self.embedding = nn.Embedding(train_config.vocab_size, train_config.hidden_size)
		self.out = nn.Dense(train_config.hidden_size*train_config.max_length, train_config.vocab_size)
	def construct(self, images, questions):
		x = self.embedding(questions)
		x = x.reshape(x.shape[0], -1)
		x = self.out(x)
		return x

In [12]:
from model.vqa_baseline import *

In [13]:
# 创建网络
network = VQABasic(train_config)

In [14]:
# 创建训练、测试网络
loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=True)
# train network
train_net = TrainNetworkWrapper(network, loss_fn, train_config)
train_net.set_train(True)
# valid network
valid_net = WithEvalCellWrapper(network)
valid_net.set_train(False)

WithEvalCellWrapper<
  (network): VQABasic<
    (VGGnet): VGG<
      (conv1): Conv2d<input_channels=3, output_channels=64, kernel_size=(3, 3), stride=(1, 1), pad_mode=same, padding=0, dilation=(1, 1), group=1, has_bias=False, weight_init=normal, bias_init=zeros, format=NCHW>
      (conv2): Conv2d<input_channels=64, output_channels=64, kernel_size=(3, 3), stride=(1, 1), pad_mode=same, padding=0, dilation=(1, 1), group=1, has_bias=False, weight_init=normal, bias_init=zeros, format=NCHW>
      (conv3): Conv2d<input_channels=64, output_channels=128, kernel_size=(3, 3), stride=(1, 1), pad_mode=same, padding=0, dilation=(1, 1), group=1, has_bias=False, weight_init=normal, bias_init=zeros, format=NCHW>
      (conv4): Conv2d<input_channels=128, output_channels=128, kernel_size=(3, 3), stride=(1, 1), pad_mode=same, padding=0, dilation=(1, 1), group=1, has_bias=False, weight_init=normal, bias_init=zeros, format=NCHW>
      (conv5): Conv2d<input_channels=128, output_channels=256, kernel_size=(3, 

### 3.2 开始训练

In [15]:
def train(train_net, valid_net, train_dataset, valid_dataset, train_config):
	# 创建文件夹
	if not os.path.exists(train_config.ckpt_save_path):
		os.mkdir(train_config.ckpt_save_path)
	
	current_step = 0
	valid_acc_max = 0.0
	valid_loss_min = np.inf
	valid_acc_model = 0
	valid_loss_model = np.inf
	for epoch_num in range(1, train_config.epoch_size+1):
		step_num = 1
		for i in train_dataset.create_dict_iterator():
			# train
			train_loss, train_acc = train_net(i['image'], i['question'], i['answer'])
			print('epoch:', epoch_num, ', step:', step_num, ' train loss =', train_loss, 'acc =', train_acc)
			step_num += 1
		# valid
		loss = []
		acc = []
		for j in valid_dataset.create_dict_iterator():
			step_loss, step_acc = valid_net(j['image'], j['question'], j['answer'])
			loss.append(step_loss.item(0).asnumpy().item())
			acc.append(step_acc.item(0).asnumpy().item())
		valid_loss = sum(loss) / len(loss)
		valid_acc = sum(acc) / len(acc)
		print('-- valid loss =', valid_loss, 'acc =', valid_acc)
		
		if valid_acc >= valid_acc_max or valid_loss < valid_loss_min:
			if valid_acc >= valid_acc_max and valid_loss < valid_loss_min:
				valid_acc_model = valid_acc
				valid_loss_model = valid_loss
				save_checkpoint(valid_net.network, train_config.checkpoint_path)
				valid_acc_max = np.max((valid_acc_max, valid_acc))
				valid_loss_min = np.min((valid_loss_min, valid_loss))
			current_step = 0
		else:
			current_step += 1
			if current_step == train_config.early_stop:
				print('early stop... min loss:', valid_loss_min, 'max acc:', valid_acc_max, end='')
				print('; validation model loss:', valid_loss_model, 'acc:', valid_acc_model)

In [16]:
train(train_net, valid_net, train_dataset, valid_dataset, train_config)

epoch: 1 , step: 1  train loss = 9.233382 acc = 0.3125
epoch: 1 , step: 2  train loss = 9.233294 acc = 0.09375
epoch: 1 , step: 3  train loss = 9.233098 acc = 0.25
epoch: 1 , step: 4  train loss = 9.2330265 acc = 0.15625
epoch: 1 , step: 5  train loss = 9.232498 acc = 0.15625
epoch: 1 , step: 6  train loss = 9.232389 acc = 0.28125
epoch: 1 , step: 7  train loss = 9.232519 acc = 0.15625
epoch: 1 , step: 8  train loss = 9.231946 acc = 0.1875
epoch: 1 , step: 9  train loss = 9.231836 acc = 0.1875
epoch: 1 , step: 10  train loss = 9.231499 acc = 0.25
epoch: 1 , step: 11  train loss = 9.231361 acc = 0.1875
epoch: 1 , step: 12  train loss = 9.231409 acc = 0.21875
epoch: 1 , step: 13  train loss = 9.231018 acc = 0.21875
epoch: 1 , step: 14  train loss = 9.230887 acc = 0.125
epoch: 1 , step: 15  train loss = 9.230423 acc = 0.25
epoch: 1 , step: 16  train loss = 9.229078 acc = 0.1875
epoch: 1 , step: 17  train loss = 9.229333 acc = 0.25
epoch: 1 , step: 18  train loss = 9.22938 acc = 0.15625
ep

In [18]:
mox.file.copy_parallel(src_url=train_config.checkpoint_path, dst_url=obs_path+train_config.checkpoint_path) 

## 4 测试模型

### 4.1 创建测试模型

In [12]:
from mindspore import load_checkpoint
from model.vqa_baseline import *

In [13]:
network = VQABasic(train_config)

In [14]:
load_checkpoint(train_config.checkpoint_path, net=network)
test_net = WithEvalCellWrapper(network)
test_net.set_train(False)

WithEvalCellWrapper<
  (network): VQABasic<
    (VGGnet): VGG<
      (conv1): Conv2d<input_channels=3, output_channels=64, kernel_size=(3, 3), stride=(1, 1), pad_mode=same, padding=0, dilation=(1, 1), group=1, has_bias=False, weight_init=normal, bias_init=zeros, format=NCHW>
      (conv2): Conv2d<input_channels=64, output_channels=64, kernel_size=(3, 3), stride=(1, 1), pad_mode=same, padding=0, dilation=(1, 1), group=1, has_bias=False, weight_init=normal, bias_init=zeros, format=NCHW>
      (conv3): Conv2d<input_channels=64, output_channels=128, kernel_size=(3, 3), stride=(1, 1), pad_mode=same, padding=0, dilation=(1, 1), group=1, has_bias=False, weight_init=normal, bias_init=zeros, format=NCHW>
      (conv4): Conv2d<input_channels=128, output_channels=128, kernel_size=(3, 3), stride=(1, 1), pad_mode=same, padding=0, dilation=(1, 1), group=1, has_bias=False, weight_init=normal, bias_init=zeros, format=NCHW>
      (conv5): Conv2d<input_channels=128, output_channels=256, kernel_size=(3, 

### 4.2 开始测试

In [15]:
def test(test_net, test_dataset):
	# test
	loss = []
	acc = []
	for i in test_dataset.create_dict_iterator():
		step_loss, step_acc = test_net(i['image'], i['question'], i['answer'])
		loss.append(step_loss.item(0).asnumpy().item())
		acc.append(step_acc.item(0).asnumpy().item())
	test_loss = sum(loss) / len(loss)
	test_acc = sum(acc) / len(acc)
	print('test loss =', test_loss, 'acc =', test_acc)

In [16]:
test(test_net, test_dataset)

test loss = 7.722210412649423 acc = 0.2080977382875606
